# Settings

In [ ]:
!git clone https://github.com/QuocThinh73/HCMAI2025.git

In [ ]:
%cd HCMAI2025

In [ ]:
!pip install open_clip_torch

In [ ]:
!pip install faiss-cpu==1.11.0

# Tải và Giải nén

In [ ]:
!gdown 12NJW4WjmygihyxR00R0XEf-ldyAUtgwh

In [ ]:
!unzip -o database.zip -d .

# Build FAISS

In [ ]:
import os
import open_clip

open_clip.list_pretrained()

In [ ]:
from models.openclip import OpenCLIP

BACKBONE = "ViT-SO400M-14-SigLIP-384"
PRETRAINED = "webli"
model = OpenCLIP(backbone=BACKBONE, pretrained=PRETRAINED)

In [ ]:
from faiss_index import Faiss

my_faiss = Faiss(model=model)

In [ ]:
model_name = f"OpenCLIP_{BACKBONE}_{PRETRAINED}"

my_faiss.build(mapping_json="database/id2path.json", model_name=model_name)

# Test

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def display_results(paths, scores=None, max_images=5):
    """Display image results in Colab."""
    n = min(len(paths), max_images)
    fig, axes = plt.subplots(1, n, figsize=(15, 3))

    if n == 1:
        axes = [axes]

    for i in range(n):
        try:
            img = Image.open(paths[i])
            axes[i].imshow(img)
            if scores is not None:
                axes[i].set_title(f"Score: {scores[i]:.4f}")
            axes[i].axis('off')
        except Exception as e:
            print(f"Error displaying image {paths[i]}: {e}")

    plt.tight_layout()
    plt.show()

In [ ]:
# Load FAISS index
embeddings_path = os.path.join("database", f"{model_name}_faiss.bin")
id2path_path = os.path.join("database", "id2path.json")

my_faiss.load(embeddings_path, id2path_path)

In [ ]:
query = "Doctor"
scores, indices, paths = my_faiss.text_search(query, top_k=10)

for i, (score, idx, path) in enumerate(zip(scores, indices, paths)):
           print(f"{i+1}. Score: {score:.4f}, Index: {idx}, Path: {path}")

display_results(paths, scores, 5)